# Chapter 2: メトロポリス・ヘイスティングス法

## 学習目標
- メトロポリス・ヘイスティングス法のアルゴリズムを理解する
- 受理確率の導出と意味を学ぶ
- 提案分布の選択とその影響を理解する
- 実装と性能評価方法を習得する
- 様々な分布からのサンプリング例を実践する

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from scipy.optimize import minimize_scalar
import warnings
warnings.filterwarnings('ignore')

plt.rcParams['font.family'] = 'DejaVu Sans'
sns.set_style("whitegrid")
np.random.seed(42)

## 2.1 メトロポリス・ヘイスティングス法の基本原理

メトロポリス・ヘイスティングス法（MH法）は、目標分布$\pi(x)$からサンプリングを行うために、以下の手順を繰り返します：

### アルゴリズム
1. 現在の状態を$x^{(t)}$とする
2. 提案分布$q(x'|x^{(t)})$から新しい状態$x'$を提案
3. 受理確率を計算：
   $$\alpha = \min\left(1, \frac{\pi(x')q(x^{(t)}|x')}{\pi(x^{(t)})q(x'|x^{(t)})}\right)$$
4. 確率$\alpha$で$x'$を受理、そうでなければ$x^{(t)}$に留まる
5. $t \leftarrow t+1$として2に戻る

### 受理確率の導出

詳細釣り合い条件を満たすために：
$$\pi(x) P(x \rightarrow x') = \pi(x') P(x' \rightarrow x)$$

遷移確率を $P(x \rightarrow x') = q(x'|x) \alpha(x, x')$ として、
$$\alpha(x, x') = \min\left(1, \frac{\pi(x')q(x|x')}{\pi(x)q(x'|x)}\right)$$

これにより詳細釣り合い条件が満たされます。

## 2.2 基本実装

In [ ]:
def metropolis_hastings(target_log_pdf, proposal_sampler, proposal_log_pdf, 
                       initial_value, n_samples, verbose=False):
    """
    メトロポリス・ヘイスティングス法の汎用実装
    
    Parameters:
    - target_log_pdf: 目標分布の対数確率密度関数
    - proposal_sampler: 提案分布からのサンプラー関数 (current_state) -> proposed_state
    - proposal_log_pdf: 提案分布の対数確率密度関数 (proposed, current) -> log_q
    - initial_value: 初期値
    - n_samples: サンプル数
    - verbose: 詳細情報の表示
    
    Returns:
    - samples: サンプル配列
    - acceptance_rate: 受理率
    - log_probs: 各サンプルの対数確率
    """
    # 初期化
    if np.isscalar(initial_value):
        samples = np.zeros(n_samples)
        dim = 1
    else:
        samples = np.zeros((n_samples, len(initial_value)))
        dim = len(initial_value)
    
    current = np.copy(initial_value)
    current_log_prob = target_log_pdf(current)
    n_accepted = 0
    log_probs = np.zeros(n_samples)
    
    for i in range(n_samples):
        # 新しい状態を提案
        proposed = proposal_sampler(current)
        proposed_log_prob = target_log_pdf(proposed)
        
        # 受理確率を計算（対数スケールで安全に計算）
        log_alpha = (proposed_log_prob + proposal_log_pdf(current, proposed) - 
                    current_log_prob - proposal_log_pdf(proposed, current))
        alpha = min(1.0, np.exp(log_alpha))
        
        # 受理/棄却を決定
        if np.random.rand() < alpha:
            current = proposed
            current_log_prob = proposed_log_prob
            n_accepted += 1
        
        if dim == 1:
            samples[i] = current
        else:
            samples[i] = current
        log_probs[i] = current_log_prob
        
        if verbose and (i + 1) % (n_samples // 10) == 0:
            print(f"Progress: {i+1}/{n_samples}, Acceptance Rate: {n_accepted/(i+1):.3f}")
    
    acceptance_rate = n_accepted / n_samples
    return samples, acceptance_rate, log_probs

## 2.3 例1：混合正規分布からのサンプリング

まず、1次元の混合正規分布からサンプリングしてみましょう。

In [ ]:
# 混合正規分布の定義
def mixture_log_pdf(x):
    """2つの正規分布の混合の対数確率密度"""
    component1 = stats.norm.logpdf(x, -2, 0.5)
    component2 = stats.norm.logpdf(x, 2, 1.0)
    # log(0.3 * exp(component1) + 0.7 * exp(component2))
    max_comp = np.maximum(component1, component2)
    return max_comp + np.log(0.3 * np.exp(component1 - max_comp) + 
                            0.7 * np.exp(component2 - max_comp))

# 対称な提案分布（ランダムウォーク）
def random_walk_sampler(current, step_size=0.5):
    return current + np.random.normal(0, step_size)

def symmetric_proposal_log_pdf(proposed, current):
    return 0.0  # 対称な提案分布の場合、比は1（対数で0）

# サンプリング実行
samples, acceptance_rate, log_probs = metropolis_hastings(
    target_log_pdf=mixture_log_pdf,
    proposal_sampler=lambda x: random_walk_sampler(x, 0.8),
    proposal_log_pdf=symmetric_proposal_log_pdf,
    initial_value=0.0,
    n_samples=10000,
    verbose=True
)

print(f"\n最終受理率: {acceptance_rate:.3f}")

In [ ]:
# 結果の可視化
def plot_mcmc_results_1d(samples, target_log_pdf, burnin=1000, title="MCMC Results"):
    fig, axes = plt.subplots(2, 3, figsize=(15, 8))
    
    # トレースプロット
    axes[0, 0].plot(samples[:2000], alpha=0.7, linewidth=0.8)
    axes[0, 0].set_title('Trace Plot (first 2000 samples)')
    axes[0, 0].set_xlabel('Iteration')
    axes[0, 0].set_ylabel('Value')
    axes[0, 0].axvline(burnin, color='red', linestyle='--', alpha=0.7, label='Burn-in')
    axes[0, 0].legend()
    
    # ヒストグラムと真の分布の比較
    axes[0, 1].hist(samples[burnin:], bins=60, density=True, alpha=0.7, 
                    color='skyblue', label='MCMC samples')
    x_range = np.linspace(samples.min(), samples.max(), 1000)
    true_density = np.exp(target_log_pdf(x_range))
    axes[0, 1].plot(x_range, true_density, 'r-', linewidth=2, label='True distribution')
    axes[0, 1].set_title('Sample Distribution vs True Distribution')
    axes[0, 1].set_xlabel('Value')
    axes[0, 1].set_ylabel('Density')
    axes[0, 1].legend()
    
    # 自己相関関数
    from statsmodels.tsa.stattools import acf
    lags = min(200, len(samples[burnin:]) // 10)
    autocorr = acf(samples[burnin:], nlags=lags, fft=True)
    axes[0, 2].plot(autocorr)
    axes[0, 2].axhline(0, color='k', linestyle='--', alpha=0.5)
    axes[0, 2].axhline(0.05, color='r', linestyle='--', alpha=0.5, label='5%')
    axes[0, 2].axhline(-0.05, color='r', linestyle='--', alpha=0.5)
    axes[0, 2].set_title('Autocorrelation Function')
    axes[0, 2].set_xlabel('Lag')
    axes[0, 2].set_ylabel('ACF')
    axes[0, 2].legend()
    
    # 累積平均
    cumulative_mean = np.cumsum(samples[burnin:]) / np.arange(1, len(samples[burnin:]) + 1)
    axes[1, 0].plot(cumulative_mean)
    true_mean = np.sum([0.3 * (-2), 0.7 * 2])  # 混合分布の理論平均
    axes[1, 0].axhline(true_mean, color='r', linestyle='--', label=f'True mean = {true_mean:.2f}')
    axes[1, 0].set_title('Cumulative Mean')
    axes[1, 0].set_xlabel('Iteration')
    axes[1, 0].set_ylabel('Mean')
    axes[1, 0].legend()
    
    # 受理率の推移
    window_size = len(samples) // 100
    running_acceptance = []
    for i in range(window_size, len(samples), window_size):
        # 簡易的な受理率計算（連続する値の変化で判定）
        window_samples = samples[i-window_size:i]
        changes = np.sum(np.diff(window_samples) != 0)
        running_acceptance.append(changes / window_size)
    
    axes[1, 1].plot(running_acceptance)
    axes[1, 1].axhline(acceptance_rate, color='r', linestyle='--', 
                       label=f'Overall: {acceptance_rate:.3f}')
    axes[1, 1].set_title('Running Acceptance Rate')
    axes[1, 1].set_xlabel('Window')
    axes[1, 1].set_ylabel('Acceptance Rate')
    axes[1, 1].legend()
    
    # QQプロット（理論分布との比較は困難なので、正規性のテスト）
    from scipy.stats import probplot
    probplot(samples[burnin:], dist="norm", plot=axes[1, 2])
    axes[1, 2].set_title('Q-Q Plot (vs Normal)')
    
    plt.suptitle(title, fontsize=16)
    plt.tight_layout()
    plt.show()

plot_mcmc_results_1d(samples, mixture_log_pdf, title="Mixture Gaussian Sampling")

## 2.4 提案分布の影響

提案分布のステップサイズが性能に与える影響を調べてみましょう。

In [ ]:
def compare_step_sizes(step_sizes, n_samples=5000):
    """
    異なるステップサイズでの性能比較
    """
    results = {}
    
    for step_size in step_sizes:
        print(f"Testing step size: {step_size}")
        
        samples, acc_rate, _ = metropolis_hastings(
            target_log_pdf=mixture_log_pdf,
            proposal_sampler=lambda x: random_walk_sampler(x, step_size),
            proposal_log_pdf=symmetric_proposal_log_pdf,
            initial_value=0.0,
            n_samples=n_samples
        )
        
        # 有効サンプルサイズの計算（自己相関を考慮）
        from statsmodels.tsa.stattools import acf
        burnin = n_samples // 5
        autocorr = acf(samples[burnin:], nlags=min(200, len(samples[burnin:])//4), fft=True)
        
        # 最初に0.05を下回るラグを見つける
        tau_int = 1
        for lag in range(1, len(autocorr)):
            if autocorr[lag] < 0.05:
                tau_int = lag
                break
        
        eff_sample_size = len(samples[burnin:]) / (2 * tau_int + 1)
        
        results[step_size] = {
            'samples': samples,
            'acceptance_rate': acc_rate,
            'autocorr_time': tau_int,
            'eff_sample_size': eff_sample_size,
            'efficiency': eff_sample_size * acc_rate  # 総合効率の指標
        }
    
    return results

# 異なるステップサイズで比較
step_sizes = [0.1, 0.5, 1.0, 2.0, 5.0]
comparison_results = compare_step_sizes(step_sizes)

# 結果の表示
print("\n=== Step Size Comparison ===")
print(f"{'Step Size':<10} {'Acc Rate':<10} {'Autocorr':<10} {'Eff Size':<12} {'Efficiency':<12}")
print("-" * 60)
for step_size in step_sizes:
    result = comparison_results[step_size]
    print(f"{step_size:<10.1f} {result['acceptance_rate']:<10.3f} "
          f"{result['autocorr_time']:<10d} {result['eff_sample_size']:<12.1f} "
          f"{result['efficiency']:<12.1f}")

In [ ]:
# ステップサイズ比較の可視化
fig, axes = plt.subplots(2, 3, figsize=(15, 10))

# 受理率 vs ステップサイズ
step_sizes_plot = list(comparison_results.keys())
acc_rates = [comparison_results[s]['acceptance_rate'] for s in step_sizes_plot]
axes[0, 0].plot(step_sizes_plot, acc_rates, 'bo-')
axes[0, 0].axhline(0.44, color='r', linestyle='--', alpha=0.7, label='Optimal (~44%)')
axes[0, 0].set_xlabel('Step Size')
axes[0, 0].set_ylabel('Acceptance Rate')
axes[0, 0].set_title('Acceptance Rate vs Step Size')
axes[0, 0].legend()
axes[0, 0].set_xscale('log')

# 自己相関時間 vs ステップサイズ
autocorr_times = [comparison_results[s]['autocorr_time'] for s in step_sizes_plot]
axes[0, 1].plot(step_sizes_plot, autocorr_times, 'go-')
axes[0, 1].set_xlabel('Step Size')
axes[0, 1].set_ylabel('Autocorrelation Time')
axes[0, 1].set_title('Autocorrelation Time vs Step Size')
axes[0, 1].set_xscale('log')

# 効率 vs ステップサイズ
efficiencies = [comparison_results[s]['efficiency'] for s in step_sizes_plot]
axes[0, 2].plot(step_sizes_plot, efficiencies, 'ro-')
axes[0, 2].set_xlabel('Step Size')
axes[0, 2].set_ylabel('Efficiency (Eff Size × Acc Rate)')
axes[0, 2].set_title('Efficiency vs Step Size')
axes[0, 2].set_xscale('log')

# トレースプロットの比較（最初の1000サンプル）
colors = ['blue', 'green', 'red', 'purple', 'orange']
for i, step_size in enumerate([0.1, 0.5, 2.0]):
    samples = comparison_results[step_size]['samples'][:1000]
    axes[1, i].plot(samples, alpha=0.7, color=colors[i])
    axes[1, i].set_title(f'Trace: Step Size = {step_size}')
    axes[1, i].set_xlabel('Iteration')
    axes[1, i].set_ylabel('Value')

plt.tight_layout()
plt.show()

## 2.5 例2：非対称な提案分布

今度は非対称な提案分布を使った例を見てみましょう。

In [ ]:
# 指数分布からのサンプリング
def exponential_log_pdf(x, rate=1.0):
    """指数分布の対数確率密度"""
    if x < 0:
        return -np.inf
    return np.log(rate) - rate * x

# 非対称な提案分布（対数正規分布）
def lognormal_proposal_sampler(current, sigma=0.5):
    """対数正規分布による提案"""
    return current * np.exp(np.random.normal(0, sigma))

def lognormal_proposal_log_pdf(proposed, current, sigma=0.5):
    """対数正規提案分布の対数確率密度"""
    if proposed <= 0 or current <= 0:
        return -np.inf
    log_ratio = np.log(proposed / current)
    return -0.5 * (log_ratio / sigma)**2 - 0.5 * np.log(2 * np.pi * sigma**2) - np.log(proposed)

# サンプリング実行
rate_param = 2.0
samples_exp, acceptance_rate_exp, _ = metropolis_hastings(
    target_log_pdf=lambda x: exponential_log_pdf(x, rate_param),
    proposal_sampler=lambda x: lognormal_proposal_sampler(x, 0.3),
    proposal_log_pdf=lambda p, c: lognormal_proposal_log_pdf(p, c, 0.3),
    initial_value=1.0,
    n_samples=10000,
    verbose=True
)

print(f"\n指数分布サンプリング受理率: {acceptance_rate_exp:.3f}")
print(f"理論平均: {1/rate_param:.3f}, サンプル平均: {np.mean(samples_exp[2000:]):.3f}")

In [ ]:
# 指数分布サンプリング結果の可視化
fig, axes = plt.subplots(2, 2, figsize=(12, 10))

burnin = 2000

# トレースプロット
axes[0, 0].plot(samples_exp[:3000], alpha=0.7)
axes[0, 0].axvline(burnin, color='red', linestyle='--', alpha=0.7, label='Burn-in')
axes[0, 0].set_title('Trace Plot')
axes[0, 0].set_xlabel('Iteration')
axes[0, 0].set_ylabel('Value')
axes[0, 0].legend()

# ヒストグラムと真の分布
axes[0, 1].hist(samples_exp[burnin:], bins=50, density=True, alpha=0.7, 
                color='lightblue', label='MCMC samples')
x_range = np.linspace(0, np.percentile(samples_exp[burnin:], 95), 1000)
true_density = rate_param * np.exp(-rate_param * x_range)
axes[0, 1].plot(x_range, true_density, 'r-', linewidth=2, label='True exponential')
axes[0, 1].set_title('Sample Distribution vs True Distribution')
axes[0, 1].set_xlabel('Value')
axes[0, 1].set_ylabel('Density')
axes[0, 1].legend()

# Q-Qプロット（指数分布と比較）
from scipy.stats import probplot
probplot(samples_exp[burnin:], dist=stats.expon, sparams=(0, 1/rate_param), plot=axes[1, 0])
axes[1, 0].set_title('Q-Q Plot vs Exponential Distribution')

# 累積分布関数の比較
sorted_samples = np.sort(samples_exp[burnin:])
empirical_cdf = np.arange(1, len(sorted_samples) + 1) / len(sorted_samples)
theoretical_cdf = 1 - np.exp(-rate_param * sorted_samples)

axes[1, 1].plot(sorted_samples, empirical_cdf, 'b-', alpha=0.7, label='Empirical CDF')
axes[1, 1].plot(sorted_samples, theoretical_cdf, 'r-', linewidth=2, label='Theoretical CDF')
axes[1, 1].set_title('CDF Comparison')
axes[1, 1].set_xlabel('Value')
axes[1, 1].set_ylabel('Cumulative Probability')
axes[1, 1].legend()

plt.tight_layout()
plt.show()

## 2.6 多変量分布への拡張

2次元の多変量正規分布からサンプリングしてみましょう。

In [ ]:
# 2次元多変量正規分布
def multivariate_normal_log_pdf(x, mu, cov):
    """多変量正規分布の対数確率密度"""
    k = len(mu)
    diff = x - mu
    
    # 数値安定性のための計算
    try:
        chol = np.linalg.cholesky(cov)
        log_det = 2 * np.sum(np.log(np.diag(chol)))
        solve = np.linalg.solve(chol, diff)
        mahalanobis_sq = np.sum(solve**2)
    except np.linalg.LinAlgError:
        return -np.inf
    
    return -0.5 * (k * np.log(2 * np.pi) + log_det + mahalanobis_sq)

# 多変量提案分布
def multivariate_proposal_sampler(current, cov_proposal):
    """多変量正規提案分布"""
    return np.random.multivariate_normal(current, cov_proposal)

def multivariate_proposal_log_pdf(proposed, current, cov_proposal):
    """多変量正規提案分布の対数確率密度（対称なので0）"""
    return 0.0

# パラメータ設定
mu_target = np.array([1.0, 2.0])
cov_target = np.array([[1.0, 0.7], [0.7, 2.0]])
cov_proposal = 0.5 * np.eye(2)

# サンプリング実行
samples_mv, acceptance_rate_mv, _ = metropolis_hastings(
    target_log_pdf=lambda x: multivariate_normal_log_pdf(x, mu_target, cov_target),
    proposal_sampler=lambda x: multivariate_proposal_sampler(x, cov_proposal),
    proposal_log_pdf=lambda p, c: multivariate_proposal_log_pdf(p, c, cov_proposal),
    initial_value=np.array([0.0, 0.0]),
    n_samples=10000,
    verbose=True
)

print(f"\n多変量正規分布サンプリング受理率: {acceptance_rate_mv:.3f}")
print(f"理論平均: {mu_target}")
print(f"サンプル平均: {np.mean(samples_mv[2000:], axis=0)}")
print(f"理論共分散:\n{cov_target}")
print(f"サンプル共分散:\n{np.cov(samples_mv[2000:].T)}")

In [ ]:
# 多変量結果の可視化
fig, axes = plt.subplots(2, 3, figsize=(15, 10))

burnin = 2000
samples_clean = samples_mv[burnin:]

# 散布図
axes[0, 0].scatter(samples_clean[:, 0], samples_clean[:, 1], alpha=0.6, s=1)
axes[0, 0].set_xlabel('X1')
axes[0, 0].set_ylabel('X2')
axes[0, 0].set_title('Scatter Plot of Samples')
axes[0, 0].set_aspect('equal')

# 等高線プロット
x1_range = np.linspace(samples_clean[:, 0].min(), samples_clean[:, 0].max(), 50)
x2_range = np.linspace(samples_clean[:, 1].min(), samples_clean[:, 1].max(), 50)
X1, X2 = np.meshgrid(x1_range, x2_range)
pos = np.dstack((X1, X2))

# 真の分布の等高線
rv = stats.multivariate_normal(mu_target, cov_target)
axes[0, 1].contour(X1, X2, rv.pdf(pos), colors='red', alpha=0.8)
axes[0, 1].scatter(samples_clean[::10, 0], samples_clean[::10, 1], alpha=0.3, s=1)
axes[0, 1].set_xlabel('X1')
axes[0, 1].set_ylabel('X2')
axes[0, 1].set_title('Samples with True Distribution Contours')

# マージナル分布
axes[0, 2].hist(samples_clean[:, 0], bins=50, density=True, alpha=0.7, label='X1 samples')
x1_theory = np.linspace(samples_clean[:, 0].min(), samples_clean[:, 0].max(), 100)
axes[0, 2].plot(x1_theory, stats.norm.pdf(x1_theory, mu_target[0], np.sqrt(cov_target[0, 0])), 
                'r-', label='X1 true')
axes[0, 2].set_title('Marginal Distribution X1')
axes[0, 2].legend()

# トレースプロット
axes[1, 0].plot(samples_mv[:3000, 0], alpha=0.7, label='X1')
axes[1, 0].plot(samples_mv[:3000, 1], alpha=0.7, label='X2')
axes[1, 0].axvline(burnin, color='red', linestyle='--', alpha=0.7, label='Burn-in')
axes[1, 0].set_title('Trace Plot')
axes[1, 0].set_xlabel('Iteration')
axes[1, 0].set_ylabel('Value')
axes[1, 0].legend()

# 第2マージナル分布
axes[1, 1].hist(samples_clean[:, 1], bins=50, density=True, alpha=0.7, label='X2 samples')
x2_theory = np.linspace(samples_clean[:, 1].min(), samples_clean[:, 1].max(), 100)
axes[1, 1].plot(x2_theory, stats.norm.pdf(x2_theory, mu_target[1], np.sqrt(cov_target[1, 1])), 
                'r-', label='X2 true')
axes[1, 1].set_title('Marginal Distribution X2')
axes[1, 1].legend()

# 相関の収束
n_points = len(samples_clean)
window_size = n_points // 100
correlations = []
for i in range(window_size, n_points, window_size):
    window_samples = samples_clean[i-window_size:i]
    corr = np.corrcoef(window_samples[:, 0], window_samples[:, 1])[0, 1]
    correlations.append(corr)

axes[1, 2].plot(correlations)
true_corr = cov_target[0, 1] / np.sqrt(cov_target[0, 0] * cov_target[1, 1])
axes[1, 2].axhline(true_corr, color='red', linestyle='--', 
                   label=f'True correlation = {true_corr:.3f}')
axes[1, 2].set_title('Running Correlation')
axes[1, 2].set_xlabel('Window')
axes[1, 2].set_ylabel('Correlation')
axes[1, 2].legend()

plt.tight_layout()
plt.show()

## 2.7 演習問題

### 問題1：ベータ分布からのサンプリング
ベータ分布 $\text{Beta}(\alpha=2, \beta=5)$ からメトロポリス・ヘイスティングス法でサンプリングしなさい。

In [ ]:
# 問題1の解答欄
def beta_log_pdf(x, alpha=2, beta=5):
    """ベータ分布の対数確率密度"""
    if x <= 0 or x >= 1:
        return -np.inf
    return (alpha - 1) * np.log(x) + (beta - 1) * np.log(1 - x)

# ここに実装してください
# ヒント：[0,1]区間に制約があるので、提案が範囲外の場合の処理が必要

pass  # 学習者が実装

### 問題2：最適な受理率の調査
1次元正規分布に対して、異なるステップサイズで受理率と効率を調べ、最適な受理率（約23%）が実際に効率的かを確認しなさい。

In [ ]:
# 問題2の解答欄
def standard_normal_log_pdf(x):
    return -0.5 * x**2 - 0.5 * np.log(2 * np.pi)

# ここに実装してください
# ヒント：複数のステップサイズで実験し、受理率と自己相関時間の関係を調べる

pass  # 学習者が実装

## まとめ

この章では、メトロポリス・ヘイスティングス法について以下を学習しました：

1. **基本アルゴリズム**：提案→受理確率計算→受理/棄却のサイクル
2. **受理確率の導出**：詳細釣り合い条件から導かれる公式
3. **提案分布の重要性**：ステップサイズが性能に大きく影響
4. **対称・非対称提案**：提案分布の選択と実装の違い
5. **多変量への拡張**：高次元分布での適用方法
6. **性能評価**：受理率、自己相関時間、有効サンプルサイズ

### 重要なポイント
- **最適受理率**：1次元で約44%、高次元で約23%
- **提案分布の調整**：問題に応じた適切な選択が重要
- **数値安定性**：対数スケールでの計算で数値誤差を防ぐ
- **診断の重要性**：トレースプロット、自己相関、収束チェック

次の章では、特別な構造を持つ多変量分布に対してより効率的なギブスサンプリングを学習します。